# Tensor network quantum control for expectation value minimisation

In this program, we take the optimised gate sequence from _vqe_from_expval_mpo.m_ and implement a state vector simulation to confirm they correctly minimising the expectation value.

In [9]:
import numpy as np
import random
import itertools
import time
from matplotlib import pyplot as plt
from qutip import *

from pauli_string_functions_module_python import *

In [31]:
#crushed Ising model (cIm) - definition, spectrum and eigenstates
n = 4
X1, Xn = 'X' + 'I' * (n - 1), 'I' * (n - 1) + 'X'
cIm_terms = tuple(list(all_single_operator_strings(n, 'Z')) + [X1, Xn] + list(local_2body_strings(n, 'XX')))
H_cIm = 0 
for j in range(len(crushed_Ising_model_terms)):
    H_cIm += operator_string_from_Pauli_string(cIm_terms[j])

cIm_spectrum, cIm_states = H_cIm.eigenstates()
cIm_GS = cIm_states[0]

[-5.45741483 -4.36501413 -3.40723125 -2.66166005 -2.18870229 -1.17631184
 -0.43074065  0.04221712  0.52704224  1.          1.61944294  1.7455712
  2.0924007   2.83797189  3.79575478  6.02667418]
-5.457414830239131


In [126]:
#importing and organising data
n = 4
bin_num = 2
ctrl_num = 8
x_optm = np.loadtxt('data/cIm_bin_num=2_n=4_test.csv', delimiter=',')
T_optm = x_optm[-1]
Dt = T_optm / bin_num
c_optm = np.reshape(x_optm[:-1], (bin_num, ctrl_num))

cX_temp, cY_temp = np.zeros([n, bin_num]), np.zeros([n, bin_num])
for i in range(n):
    cX_temp[i] = c_optm[:, 2 * i]
    cY_temp[i] = c_optm[:, 2 * i + 1]

#swapping sign and order of cX an cY
cX_binned, cY_binned = np.zeros([n, bin_num]), np.zeros([n, bin_num])
for i in range(n):
    cX_binned[i] = np.flip(-cX_temp[i])
    cY_binned[i] = np.flip(-cY_temp[i])
#row represents the bin, collumn represents the qubit on which X or Y act

In [128]:
#state vector simulation parameters and time-dependent pulse definition
time_steps = 10000
times = np.linspace(0, T_optm, time_steps)

cX, cY = np.zeros([n, time_steps]), np.zeros([n, time_steps])
for i in range(n):
    count = 1
    for j in range(time_steps):
        if times[j] > count * Dt:
            count += 1
        cX[i, j] = cX_binned[i, count - 1]
        cY[i, j] = cY_binned[i, count - 1]


array([[-0.14738779, -0.14738779, -0.14738779, ...,  0.44361568,
         0.44361568,  0.44361568],
       [ 0.49118989,  0.49118989,  0.49118989, ..., -0.2586076 ,
        -0.2586076 , -0.2586076 ],
       [-0.34625889, -0.34625889, -0.34625889, ...,  0.09448364,
         0.09448364,  0.09448364],
       [ 0.36010245,  0.36010245,  0.36010245, ..., -0.33064405,
        -0.33064405, -0.33064405]])

In [ ]:
psi0 = tensor([basis(2, 0)] * n)

In [124]:
cX_rev

array([[-0.14738779,  0.44361568],
       [ 0.49118989, -0.2586076 ],
       [-0.34625889,  0.09448364],
       [ 0.36010245, -0.33064405]])

1.7735192001799